# Keras Reloading Experiment 
Test the reloading of weight for the model from previously trained module from other molecules. This is only a temporary solution since our current Keras Framework only supports training the same molecule, which might be good enough for the MD simulation. New Frameworks will replace current ones. 
<br/>

# 0. Build the Keras Model. 
Construct the Keras Model for a molecule with different configurations. 

In [15]:
from src_nogrd import at_idx_map_generator


import os
import numpy as np 

import keras 
from keras.layers import Input, Dense, merge
from keras.models import Model, Sequential

from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [16]:
from ase import Atoms
import Utils.DirNav 
import tests.test_Utils
project_dir = Utils.DirNav.get_project_dir() 

leucine, _, Gparam_dict = tests.test_Utils.import_glycine_mol(filename='leucine.xyz')

Atoms(symbols='C2HC2HCHNHCH6O2H4', pbc=False)


In [17]:
type(leucine)

ase.atoms.Atoms

In [18]:
import src_nogrd

at_idx_map = src_nogrd.at_idx_map_generator(leucine)

In [19]:
at_idx_map

{'H': array([ 2,  5,  7,  9, 11, 12, 13, 14, 15, 16, 19, 20, 21, 22]),
 'O': array([17, 18]),
 'C': array([ 0,  1,  3,  4,  6, 10]),
 'N': array([8])}

In [20]:
nUnit = 34 #int(os.environ['unit_number'])
def Nguyen_Widrow_init(shape, dtype=None):
    '''Nguyen-Widrow initialization for weights and biases
    Initialize the weight and biases for each layer.
    
            Args:
                shape: ???? (Guess: Shape of the layer to create)
            
            Output:
                [w,b]
                w: represents an array of weight
                b: represents an array of bias
    
    Comment:
    This is the function provided by Nguyen's original code. 
    Junmian Zhu did the documentation. It is not 100% sure about the 
    algorithm used here. 
    
    '''
    n_input, n_unit = shape[0], shape[1]  # n_input: number of units of prev. layer without bias 
    w_init = np.random.rand(n_input, n_unit) *2 -1
    norm = 0.7 * n_unit ** (1. / n_input)
    # normalize 
    w = norm * w_init/np.sqrt(np.square(w_init).sum(axis=0).reshape(1, n_unit))
    if n_unit>1:
        b = norm * np.linspace(-1,1,n_unit) * np.sign(w[0,:]) 
    else: 
        b = np.zeros((n_unit,)) 
    return [w, b]

def create_base_network(input_dim, name):
    '''3-layer base network to be shared among atoms of the same species 
    '''    
    seq = Sequential()
    seq.add(Dense(nUnit, input_shape=(input_dim,), activation='tanh'))
    shape0 = (seq.layers[0].input_shape[1], seq.layers[0].output_shape[1]) 
    seq.layers[0].set_weights(Nguyen_Widrow_init(shape0))
    
    seq.add(Dense(nUnit, activation='tanh'))
    shape1 = (seq.layers[1].input_shape[1], seq.layers[1].output_shape[1]) 
    seq.layers[1].set_weights(Nguyen_Widrow_init(shape1))
    
    seq.add(Dense(nUnit, activation='tanh'))
    shape2 = (seq.layers[2].input_shape[1], seq.layers[2].output_shape[1]) 
    seq.layers[2].set_weights(Nguyen_Widrow_init(shape2))

    seq.add(Dense(1, activation='linear'))
    shape3 = (seq.layers[3].input_shape[1], seq.layers[3].output_shape[1]) 
    seq.layers[3].set_weights(Nguyen_Widrow_init(shape3)) 
    
    seq.name = name
    return seq

def main_network(input_dim_list):
    '''
    Main neural network with shared sub-structures.
    Currently it supports a sub-network with additional features. 
    '''
    type_nb = len(input_dim_list)
    element_network = {}
    inp = {'element':{}, 'rc':{}}
    
    element_processed = {}  # Will keep the input of the atomic symmetry function
    
    
    
    for at_type in at_idx_map:
        dims = input_dim_list[at_type]
        # Create a sub-network for each element. 
        element_network[at_type] = create_base_network(dims[0], at_type + '-subnet')

        element_processed[at_type] = {}
        inp['element'][at_type] = {}
        
        for atA in at_idx_map[at_type]:
            # input preparation
            inp['element'][at_type][atA] = Input(shape=(dims[0],),dtype='float32', name=at_type+'-'+str(atA)+'-ele')

            element_processed[at_type][atA] = element_network[at_type](inp['element'][at_type][atA])
    
    
    # Define the output (predicted total energy) as a sum of all atomic energies
    element_outputs = [element_processed[u][v] for u in element_processed.keys() for v in element_processed[u].keys()]
    total_outputs = element_outputs #+ rc_processed
    main_output = keras.layers.add(total_outputs)    

    # Organize the input structure. Keras will organize the input according to the input sequence. 
    inputs = []
    
    for at_type in at_idx_map:    
        dims = input_dim_list[at_type]
        
        for atA in at_idx_map[at_type]:
            inputs.append(inp['element'][at_type][atA])
    
    # Finalizing building the main network model. 
    main_network = Model(inputs, main_output)  

    return main_network

In [21]:
n_symm_func = 276 
#input_dim_list = [(len(at_idx_map[at]) ,nSymFunc, nb_feat) for at in at_idx_map.keys()]

input_dim_list = {}
for at_type in at_idx_map:
    input_dim_list[at_type] = (n_symm_func, 1)

In [22]:
model = main_network(input_dim_list)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
H-2-ele (InputLayer)            (None, 276)          0                                            
__________________________________________________________________________________________________
H-5-ele (InputLayer)            (None, 276)          0                                            
__________________________________________________________________________________________________
H-7-ele (InputLayer)            (None, 276)          0                                            
__________________________________________________________________________________________________
H-9-ele (InputLayer)            (None, 276)          0                                            
__________________________________________________________________________________________________
H-11-ele (

# 1. Reload the Element Net weight from Trained Glycine Molecule Data. 

In [26]:
save_dir = os.path.join(project_dir, "model", 'glycine_opt')
h_subnet_weight = os.path.join(save_dir, 'H-subnet-weight.h5')
model.get_layer("H-subnet").load_weights(h_subnet_weight)

## Result:
_SUCCESS_ Load the weight from previously trained network. 